#  Practica de limpieza de texto con pandas - Solución

#### Por Jose R. Zapata - https://joserzapata.github.io/


> **Objetivo**: practicar métodos de `pandas.Series.str` para limpiar y normalizar texto en DataFrames.
> Cada ejercicio incluye un **Enunciado** y una celda donde se ve el resultado de la solución.



# Copie su código aca
import re
import numpy as np

def parse_talla(s):
    """Extrae talla en letra (talla_std) y talla numérica (talla_num) desde una descripción."""
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return (None, None)
    text = str(s).strip()
    text_up = text.upper()
    # Primera: buscar número (ej: 42, 36, 10-12 -> tomar el primer número encontrado)
    m_num = re.search(r'({1,3})', text)
    talla_num = int(m_num.group(1)) if m_num else None

    # Buscar tallas en letras: XS, S, M, L, XL, XXL o indicación de talla única (TU)
    talla_std = None
    # patrón directo (captura XS, XXL, XL, S, M, L)
    m_letter = re.search(r'(XS|XXL|XL|S|M|L)', text_up)
    if m_letter:
        talla_std = m_letter.group(1)
    else:
        # patrones como 'TALLA: ÚNICA', 'Talla Unica', '(talla Única)' o 'TU'
        if re.search(r'TALLA*[:]?*(ÚNICA|UNICA|TÚ|TU)|TU|ÚNICA|UNICA', text_up):
            talla_std = 'TU'
    # Normalizar hyphens y espacios
    if isinstance(talla_std, str):
        talla_std = talla_std.replace('-', '').strip().upper()
    # Asegurar que queda en el conjunto permitido
    allowed = {'XS','S','M','L','XL','XXL','TU'}
    if talla_std not in allowed:
        # si no hay talla letras válida, dejar None
        talla_std = talla_std if talla_std in allowed else None

    return (talla_std, talla_num)

# Aplicar la función y crear columnas en df_productos
df_productos[['talla_std','talla_num']] = df_productos['producto'].apply(lambda s: pd.Series(parse_talla(s)))
df_productos
- Documentación de `pandas.Series.str`: https://pandas.pydata.org/docs/reference/series.html#string-handling


## Instalación de dependencias

instala e importa las librerias que necesites para ejecutar los ejercicios, por ejemplo para instalar Jupyter usa:

```bash
uv add jupyter
```


In [2]:
# Copie su código aca
import pandas as pd
import unicodedata
import re, numpy as np

In [3]:
# Función para eliminar acentos (maneja None/NaN)
def eliminar_acentos(s):
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return s
    # Normalizar a NFKD y filtrar marcas diacríticas
    try:
        nkfd = unicodedata.normalize("NFKD", str(s))
        return "".join(c for c in nkfd if not unicodedata.combining(c))
    except Exception as e:
        # En caso de algún valor inesperado, devolver el original
        return s


# Prueba rápida (no imprime en el notebook al insertar, pero sirve para quien ejecute la celda)
_test_vals = ["café", "árbol", "Ñandú", None, "año"]
_test_res = [eliminar_acentos(x) for x in _test_vals]
# _test_res -> ['cafe', 'arbol', 'Nandu', None, 'ano']

In [4]:
import pandas as pd

## Conjuntos de datos de juguete
Creamos 4 DataFrames para practicar distintas tareas de limpieza: texto general, tweets, productos y personas.

In [5]:
## Texto general
df_texto = pd.DataFrame(
    {
        "texto": [
            "  ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  ",
            "Pandas > numpy? 🤔  Email: persona@example.com   ",
            "Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan",
            "Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123",
            "      TABLAS\t, \n espacios   y saltos de línea.\r\n",
            "Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44",
            "Dirección: Cll 10 # 5-20; Medellín. Barrio: La América",
            "Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",
        ]
    }
)

## Tweets
df_tweets = pd.DataFrame(
    {
        "tweet": [
            "RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python",
            "¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊",
            "Probando cosas en Jupyter... sin link ni hashtag",
            "@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml",
            "¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data",
        ]
    }
)

## Productos y precios
df_productos = pd.DataFrame(
    {
        "producto": [
            "Camisa talla M",
            "Pantalón-XL",
            "Zapato, Talla: 42",
            "Blusa s",
            "Medias 10-12",
            "Polo Talla l",
            "Vestido - 36",
            "Sombrero (talla Única)",
        ],
        "precio": [
            "$1.234,50",
            "USD 45",
            "30,00 €",
            "25.000",
            "$ 0",
            "S/. 120.90",
            "COP 9.990",
            "AR$ 2.550,00",
        ],
    }
)

## Personas, respuestas y direcciones
df_personas = pd.DataFrame(
    {
        "nombre": [
            "ana María LOPEZ",
            "Juan  perez",
            "Ñandú   Gómez",
            "Miguel (Soporte)",
            "  MÓNICA de la CRUZ  ",
            "luis-delgado",
        ],
        "categoria": ["Sí", "si", "SI ", "No", "—", None],
        "direccion": [
            "Calle 45 # 12-34, Bogotá",
            "Av. Siempre Viva 742 - Lima",
            "Cll. 10 No. 5-20 Medellín",
            "Cra 7a # 45-60, Bogotá",
            "Av. 9 #12-34  Cali",
            "Av. Insurgentes Sur 1234, CDMX",
        ],
        "id_raw": [
            "abc-0001",
            "abc 001",
            "ABC0002",
            "Abc_003",
            "abc-00004",
            "ABC-0005",
        ],
    }
)

## Inspección rápida

- Explora los cuatro DataFrames
- Identifica qué columnas requieren **limpieza textual** y por qué (espacios, acentos, emojis, URLs, etc.).
- Escribe un breve comentario (como comentario en la celda) con tus observaciones.

In [6]:
# Copie su código aca
df_texto.info()
df_tweets.info()
df_productos.info()
df_personas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   producto  8 non-null      object
 1   precio    8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     6 n

## 1) Normaliza a minúsculas

En el DataFrame `df_texto`, crea una nueva columna `texto_min` que contenga el texto de la columna `texto` en minúsculas.

In [7]:
# Copie su código aca
df_texto["texto_min"] = df_texto["texto"].str.lower()
df_texto[["texto", "texto_min"]]

,texto,texto_min
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,¡hola mundo!!! esto es un ejemplo: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,"TABLAS\t, \n espacios y saltos de líne...","tablas\t, \n espacios y saltos de líne..."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,teléfono: (57) 300-123-45-67; whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,dirección: cll 10 # 5-20; medellín. barrio: la...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 2) Eliminar Espacios

Genera  la columna `texto_espacios` eliminando espacios al inicio/fin

In [8]:
# Copie su código aca
df_texto["texto_espacios"] = df_texto["texto_min"].str.strip()
df_texto[["texto_espacios", "texto_min"]]

,texto_espacios,texto_min
0,¡hola mundo!!! esto es un ejemplo: visita h...,¡hola mundo!!! esto es un ejemplo: visita...
1,pandas > numpy? 🤔 email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano; es buenísimo!!! #...,me gusta el café colombiano; es buenísimo!!! #...
3,oferta!!! 3x2 en jabón líquido 500ml - código:...,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,"tablas\t, \n espacios y saltos de línea.","tablas\t, \n espacios y saltos de líne..."
5,teléfono: (57) 300-123-45-67; whatsapp +57 300...,teléfono: (57) 300-123-45-67; whatsapp +57 300...
6,dirección: cll 10 # 5-20; medellín. barrio: la...,dirección: cll 10 # 5-20; medellín. barrio: la...
7,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 3) Puntuación

Crea `texto_sin_punct` removiendo puntuación y símbolos, conservando letras y espacios.

In [9]:
# Copie su código aca
df_texto["texto_sin_punct"] = df_texto["texto_espacios"].str.replace(
    r"[^\w\s]", "", regex=True
)
df_texto["texto_sin_punct"]

0    hola mundo  esto es   un ejemplo visita httpsm...
1              pandas  numpy   email personaexamplecom
2    me gusta el café colombiano es buenísimo café ...
3       oferta 3x2 en jabón líquido 500ml  código a123
4             tablas\t \n espacios   y saltos de línea
5     teléfono 57 3001234567 whatsapp 57 300 222 33 44
6     dirección cll 10  520 medellín barrio la américa
7                        emoji test  símbolos  y otros
Name: texto_sin_punct, dtype: object

## 4) Acentos

Elimina los acentos de las vocales del texto `df_texto` y grábalo en una nueva columna `texto_sin_acentos`.

In [10]:
# Copie su código aca
def eliminar_acentos(text: str) -> str:
    MAP_VOCALES = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
        "Á": "A",
        "É": "E",
        "Í": "I",
        "Ó": "O",
        "Ú": "U",
        "Ü": "U",
    }
    translate = str.maketrans(MAP_VOCALES)
    text = text.translate(translate)
    return text


df_texto["texto_sin_acentos"] = df_texto["texto_sin_punct"].apply(eliminar_acentos)
df_texto["texto_sin_acentos"]

0    hola mundo  esto es   un ejemplo visita httpsm...
1              pandas  numpy   email personaexamplecom
2    me gusta el cafe colombiano es buenisimo cafe ...
3       oferta 3x2 en jabon liquido 500ml  codigo a123
4             tablas\t \n espacios   y saltos de linea
5     telefono 57 3001234567 whatsapp 57 300 222 33 44
6     direccion cll 10  520 medellin barrio la america
7                        emoji test  simbolos  y otros
Name: texto_sin_acentos, dtype: object

## 5) Emojis

Crear la columna `texto_sin_emoji` quitando emojis, pictogramas y símbolos gráficos comunes.

In [11]:
# Copie su código aca
quitar_emoji_pattern = re.compile(
    "[\U0001f300-\U0001faff"  # emoticons
    "\U00002700-\U000027bf"  # symbols & pictographs, transport & map symbols
    "\U0001f1e0-\U0001f1ff]+",  # flags (iOS)
    flags=re.UNICODE,
)
df_texto["texto_sin_emoji"] = df_texto["texto_espacios"].apply(
    lambda s: quitar_emoji_pattern.sub("", s)
)
df_texto["texto_sin_emoji"]

0    ¡hola mundo!!!  esto es   un ejemplo: visita h...
1         pandas > numpy?   email: persona@example.com
2    me gusta el café colombiano; es buenísimo!!! #...
3    oferta!!! 3x2 en jabón líquido 500ml - código:...
4           tablas\t, \n espacios   y saltos de línea.
5    teléfono: (57) 300-123-45-67; whatsapp +57 300...
6    dirección: cll 10 # 5-20; medellín. barrio: la...
7                emoji test: ️‍, símbolos ©®™ y otros…
Name: texto_sin_emoji, dtype: object

## 6) Extrae emails y URLs

En `df_texto`, crea las columnas `email` y `urls` extrayendo emails y URLs del texto original.

**Patrones sugeridos:**
- Email: `[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+`
- URL: `https?://\S+`

In [12]:
# Copie su código aca
extraer_email = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
url_url = r"https?://[^\s]+"
df_texto["email"] = df_texto["texto"].apply(lambda s: re.findall(extraer_email, s))
df_texto["urls"] = df_texto["texto"].apply(lambda s: re.findall(url_url, s))
df_texto[["texto", "email", "urls"]]

,texto,email,urls
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,[],[https://miweb.com]
1,Pandas > numpy? 🤔 Email: persona@example.com,[persona@example.com],[]
2,Me gusta el café colombiano; es buenísimo!!! #...,[],[]
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,[],[]
4,"TABLAS\t, \n espacios y saltos de líne...",[],[]
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,[],[]
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,[],[]
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",[],[]


## 7) Hashtags, menciones y URLs en tweets

En `df_tweets`, crea las columnas `hashtags`, `mentions` y `urls`
extrayendo hashtags, menciones y URLs del texto del tweet.

In [13]:
# Copie su código aca
extraer_hashtags = r"#\w+"
extraer_mentions = r"@\w+"
extraer_urls = r"https?://[^\s]+"
df_tweets["hashtags"] = df_tweets["tweet"].str.findall(extraer_hashtags)
df_tweets["mentions"] = df_tweets["tweet"].str.findall(extraer_mentions)
df_tweets["urls"] = df_tweets["tweet"].str.findall(extraer_urls)

## 8) ¿Es retuit? Limpia el prefijo

Crea la columna `es_rt` (booleano) si el tweet comienza con `RT`. Luego genera la columna `tweet_sin_rt` quitando el prefijo `RT @usuario:` del inicio.

In [14]:
# Copie su código aca
df_tweets["es_rt"] = df_tweets["tweet"].str.startswith("RT")
df_tweets["tweet_sin_rt"] = df_tweets["tweet"].str.replace(
    r"^RT\s+@\w+:\s*", "", regex=True
)

## 9) `tweet_limpio`

Crea `tweet_limpio` removiendo URLs, menciones y hashtags; además, elimina la puntuación sobrante, quita espacios y pasa a minúsculas.

In [15]:
# Copie su código aca
texto_base = (
    df_tweets["tweet_sin_rt"] if "tweet_sin_rt" in df_tweets else df_tweets["tweet"]
)
limpio = texto_base.str.replace(r"https?://\S+", " ", regex=True)  # Remover URLs
limpio = limpio.str.replace(
    r"@\w+|#\w+", " ", regex=True
)  # Remover menciones y hashtags
limpio = limpio.str.replace(
    r"[^\w\sáéíóúÁÉÍÓÚñÑ]", " ", regex=True
)  # Remover puntuación
limpio = (
    limpio.str.replace(r"\s+", " ", regex=True).str.strip().str.casefold()
)  # Espacios y minúsculas
df_tweets["tweet_limpio"] = limpio

## 10) Talla en `df_productos`

Extrae y estandariza la talla:
- Letras: `XS, S, M, L, XL, XXL`, o `TU` (talla única).
- Números: captura como `talla_num`.
Crea la columna`talla_std` (letras) y `talla_num` (numérica).

In [16]:
# Copie su código aca
pat_letra = r"\b(XS|S|M|L|XL|XXL|XXS|XXXL|TU|TALLA ÚNICA|ÚNICA|UNICA|U)\b"
pat_num = r"\b(\d{1,3})\b"

MAPEO_TALLAS = {"XXS": "XS", "XXXL": "XXL"}


def extraer_talla(row):
    txt = str(row["producto"])
    m1 = re.search(pat_letra, txt)
    m2 = re.search(pat_num, txt)
    talla_std = None
    talla_num = None

    if m1:
        val = m1.group(1).lower()
        talla_std = "TU" if val in {"única", "unica", "u"} else val.upper()
        talla_std = MAPEO_TALLAS.get(talla_std, talla_std)

    if m2:
        talla_num = float(m2.group(1)) if m2.group(1) is not None else None
        if talla_num and (talla_num < 30 or talla_num > 60):
            talla_num = None

    return pd.Series({"talla_std": talla_std, "talla_num": talla_num})


df_productos[["talla_std", "talla_num"]] = df_productos.apply(extraer_talla, axis=1)
print(df_productos)

                 producto        precio talla_std  talla_num
0          Camisa talla M     $1.234,50         M        NaN
1             Pantalón-XL        USD 45        XL        NaN
2       Zapato, Talla: 42       30,00 €       NaN       42.0
3                 Blusa s        25.000      None        NaN
4            Medias 10-12           $ 0      None        NaN
5            Polo Talla l    S/. 120.90      None        NaN
6            Vestido - 36     COP 9.990       NaN       36.0
7  Sombrero (talla Única)  AR$ 2.550,00      None        NaN


## 11) Extraer de precios

Convierte la columna `precio` a numérico (`precio_num`) independiente del formato (`$1.234,50`, `USD 45`, `30,00 €`, `COP 9.990`, etc.).

In [17]:
# Copie su código aca
def convertir_precio(text):
    text = str(text).strip()

    text = re.sub(r"[A-Za-z]", "", text)
    text = text.strip()

    if "," in text and "." in text:
        if text.rfind(",") > text.rfind("."):
            text = text.replace(".", "").replace(",", ".")
        else:
            text = text.replace(",", "")
    elif "," in text:
        if text.count(",") == 1 and len(text.split(",")[1]) <= 2:
            text = text.replace(",", ".")
        else:
            text = text.replace(",", "")

    text = re.sub(r"[^\d.]", "", text)

    try:
        return float(text) if text else None
    except ValueError:
        return None


df_productos["precio_num"] = df_productos["precio"].apply(convertir_precio)
print(df_productos)

                 producto        precio talla_std  talla_num  precio_num
0          Camisa talla M     $1.234,50         M        NaN     1234.50
1             Pantalón-XL        USD 45        XL        NaN       45.00
2       Zapato, Talla: 42       30,00 €       NaN       42.0     3000.00
3                 Blusa s        25.000      None        NaN       25.00
4            Medias 10-12           $ 0      None        NaN        0.00
5            Polo Talla l    S/. 120.90      None        NaN         NaN
6            Vestido - 36     COP 9.990       NaN       36.0        9.99
7  Sombrero (talla Única)  AR$ 2.550,00      None        NaN     2550.00


## 12) Filtrado por texto

Filtra el DataFrame `df_productos` para mostrar solo filas cuyo `producto` contenga **camisa** o **pantalón**, ignorando acentos y mayúsculas/minúsculas.

In [18]:
# Copie su código aca
mask = (
    df_productos["producto"]
    .str.lower()
    .apply(eliminar_acentos)
    .str.contains("camisa|pantalon|chaqueta")
)

df_filtrado = df_productos[mask]
print(df_filtrado)

         producto     precio talla_std  talla_num  precio_num
0  Camisa talla M  $1.234,50         M        NaN      1234.5
1     Pantalón-XL     USD 45        XL        NaN        45.0


## 13) Limpieza de nombres propios

En el DataFrame `df_personas`, crea `nombre_limpio` en `df_personas`:
- Quita paréntesis y su contenido.
- Reemplaza guiones por espacio y elimina espacios extra.
- Aplica *title case* y conserva conectores (`de`, `del`, `la`, `y`) en minúscula.

In [19]:
# Copie su código aca
def limpiar_nombre(s):
    s = re.sub(r"\([^)]*\)", " ", str(s))
    s = s.replace("-", " ")
    s = re.sub(r"\s+", " ", s).strip()
    s = s.title()
    for w in [" De ", " Del ", " La ", " Y "]:
        s = s.replace(w, w.lower())
    return s


df_personas["nombre_limpio"] = df_personas["nombre"].apply(limpiar_nombre)
print(df_personas[["nombre", "nombre_limpio"]])

                  nombre      nombre_limpio
0        ana María LOPEZ    Ana María Lopez
1            Juan  perez         Juan Perez
2          Ñandú   Gómez        Ñandú Gómez
3       Miguel (Soporte)             Miguel
4    MÓNICA de la CRUZ    Mónica de la Cruz
5           luis-delgado       Luis Delgado


## 14) Normaliza respuestas categóricas

En el DataFrame `df_personas`, convierte `categoria` en booleano `categoria_bool` donde cualquier variante de **sí** (con/ sin tilde) sea `True`; el resto `False`.

In [25]:
# Copie su código aca
def a_bool_si(s):
    if s is None:
        return False
    val = str(s).strip().casefold()
    return val in {"si", "sí", "si.", "sí."}


df_personas["categoria_bool"] = df_personas["categoria"].apply(a_bool_si)

## 15) Ciudad desde la dirección

En el DataFrame `df_personas`, Extrae la **ciudad** en una columna `ciudad`

In [28]:
# Copie su código aca
def extraer_ciudad(s):
    s = str(s)

    match = re.search(
        r"(?:en|de|de\s+|[|-]\s*)([A-Za-záéíóúÁÉÍÓÚ\s]+?)(?:\s*[,|()]|$)", s
    )

    if match:
        ciudad = match.group(1).strip()
        ciudad = re.sub(r"\s+", " ", ciudad).title()
        return ciudad

    return None

df_personas["ciudad"] = df_personas["direccion"].apply(extraer_ciudad)

## 16) Normaliza y valida IDs

En el DataFrame `df_personas`, a partir de `id_raw`, crea la columna `id_norm` con formato `ABC-0001` (tres letras + guion + 4 dígitos) y `id_valido` (True/False).

In [29]:
# Copie su código aca
def normalizar_id(valor):
    patron = re.search(r"([A-Za-z]{3})\s*[-_ ]?\s*(\d+)", str(valor))
    if not patron:
        return None
    letras = patron.group(1).upper()
    numero = int(patron.group(2))
    return f"{letras}-{numero:04d}"


df_personas["id_normalizado"] = df_personas["id_raw"].apply(normalizar_id)
df_personas["id_valido"] = df_personas["id_normalizado"].str.match(r"^[A-Z]{3}-\d{4}$")


## 17) Mini *pipeline* de limpieza general

Escribe una función `limpia_basica(s)` que: ponga el texto en minuscula, quite acentos, URLs, menciones y hashtags, puntuación, dígitos y colapse espacios. Aplícala a `df_texto['texto']` y guarda en `texto_limpio_final`.

In [30]:
# Copie su código aca
def limpia_basica(s):
    if s is None:
        return s
    s = str(s).lower()
    MAP_VOCALES = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "ü": "u",
    }
    translate = str.maketrans(MAP_VOCALES)
    s = s.translate(translate)
    s = re.sub(r"https?://\S+", " ", s)
    s = re.sub(r"[@#]\w+", " ", s)
    s = re.sub(r"[^a-z\sñ]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


df_texto["texto_limpio_final"] = df_texto["texto"].apply(limpia_basica)

## 18)¿Qué tanto cambió el texto?

En el dataframe `df_texto`, 

Cual es la diferencia entre la longitud original (`len_raw`) y la longitud tras la limpieza (`len_clean`)?

In [33]:
# Copie su código aca
df_texto["longitud_original"] = df_texto["texto"].str.len()
df_texto["longitud_diferenciada"] = df_texto["texto_limpio_final"].str.len()
df_texto["caracteres_eliminados"] = (df_texto["longitud_original"] - df_texto["longitud_diferenciada"])
df_texto[["texto", "texto_limpio_final", "longitud_original", "longitud_diferenciada"]]

,texto,texto_limpio_final,longitud_original,longitud_diferenciada
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,hola mundo esto es un ejemplo visita,84,36
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com,48,30
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el cafe colombiano es buenisimo,66,40
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta x en jabon liquido ml codigo a,52,37
4,"TABLAS\t, \n espacios y saltos de líne...",tablas espacios y saltos de linea,48,33
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,telefono whatsapp,56,17
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,direccion cll medellin barrio la america,54,40
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros,42,27


### Guardar de resultados

Guarda el dataframe df_texto en parquet con `df_texto.to_parquet('archivo.parquet', index=False)`.

**Phd. Jose R. Zapata**
- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)